We recommend using tf.keras as a high-level API for building neural networks. That said, most TensorFlow APIs are usable with eager execution.

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

In [4]:
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False


# Layers: common sets of useful operations
Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as a well as easy ways for you to write your own application-specific layers either from scratch or as the composition of existing layers.

TensorFlow includes the full Keras API in the tf.keras package, and the Keras layers are very useful when building your own models.

In [5]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

The full list of pre-existing layers can be seen in the documentation. It includes Dense (a fully-connected layer), Conv2D, LSTM, BatchNormalization, Dropout, and many others.



In [6]:
#to use a layer simply call it
layer(tf.zeros([10,5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [7]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.12312716, -0.46926773, -0.61046594, -0.09164572, -0.58611125,
          0.14086932, -0.21640775, -0.60595524, -0.57446265, -0.04772311],
        [-0.36067256,  0.16285998,  0.56271404,  0.34950137, -0.05884337,
          0.47290796,  0.3865903 ,  0.59110814, -0.3894484 ,  0.35431135],
        [ 0.12434042,  0.30291653, -0.6183433 , -0.61861324,  0.5711815 ,
          0.57255894, -0.51989836, -0.09716266, -0.08065951,  0.624551  ],
        [ 0.10940671, -0.23481566,  0.01396579,  0.3972823 ,  0.5553686 ,
         -0.22788039,  0.06920558, -0.3318818 , -0.10051411, -0.1907813 ],
        [ 0.5980931 ,  0.529808  , -0.3803801 ,  0.041879  ,  0.3003037 ,
          0.55069286,  0.0158357 ,  0.60725576,  0.31527072, -0.30037892]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [8]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.12312716, -0.46926773, -0.61046594, -0.09164572, -0.58611125,
          0.14086932, -0.21640775, -0.60595524, -0.57446265, -0.04772311],
        [-0.36067256,  0.16285998,  0.56271404,  0.34950137, -0.05884337,
          0.47290796,  0.3865903 ,  0.59110814, -0.3894484 ,  0.35431135],
        [ 0.12434042,  0.30291653, -0.6183433 , -0.61861324,  0.5711815 ,
          0.57255894, -0.51989836, -0.09716266, -0.08065951,  0.624551  ],
        [ 0.10940671, -0.23481566,  0.01396579,  0.3972823 ,  0.5553686 ,
         -0.22788039,  0.06920558, -0.3318818 , -0.10051411, -0.1907813 ],
        [ 0.5980931 ,  0.529808  , -0.3803801 ,  0.041879  ,  0.3003037 ,
          0.55069286,  0.0158357 ,  0.60725576,  0.31527072, -0.30037892]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

# Implementing custom layers
The best way to implement your own layer is extending the tf.keras.Layer class and implementing: * __ init__ , where you can do all input-independent initialization * build, where you know the shapes of the input tensors and can do the rest of the initialization * call, where you do the forward computation

Note that you don't have to wait until build is called to create your variables, you can also create them in __init__. However, the advantage of creating them in build is that it enables late variable creation based on the shape of the inputs the layer will operate on. On the other hand, creating variables in __init__ would mean that shapes required to create the variables will need to be explicitly specified.

In [14]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
    
    def build(self, input_shape):
        self.kernel = self.add_variable("kernel",
                                       shape = [int(input_shape[-1]),
                                               self.num_outputs])
        
    def call(self,input):
        return tf.matmul(input, self.kernel)
layer = MyDenseLayer(10)

In [15]:
_ = layer(tf.zeros([10,5])) #calling the layer 'builds it'

Instructions for updating:
Please use `layer.add_weight` method instead.


In [16]:
print([var.name for var in layer.trainable_variables])


['my_dense_layer/kernel:0']


Overall code is easier to read and maintain if it uses standard layers whenever possible, as other readers will be familiar with the behavior of standard layers. If you want to use a layer which is not present in tf.keras.layers, consider filing a github issue or, even better, sending us a pull request!

# Models: Composing layers
Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut. Layers can be nested inside other layers.

Typically you inherit from keras.Model when you need the model methods like: Model.fit,Model.evaluate, and Model.save (see Custom Keras layers and models for details).

One other feature provided by keras.Model (instead of keras.layers.Layer) is that in addition to tracking variables, a keras.Model also tracks its internal layers, making them easier to inspect.

For example here is a ResNet block:

In [17]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters
        
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1,1))
        self.bn2a = tf.keras.layers.BatchNormalization()
           
        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()
        
        self.conv2c = tf.keras.layers.Conv2D(filters3, (1,1))
        self.bn2c = tf.keras.layers.BatchNormalization()
    
    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x,training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2c(x)
        x = self.bn2c(x, training=training)
        
        x += input_tensor
        
        return tf.nn.relu(x)
    
block = ResnetIdentityBlock(1, [1,2,3])    

In [18]:
_ = block(tf.zeros([1,2,3,3]))

In [20]:
block.layers

In [21]:
block.summary()

Model: "resnet_identity_block"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  4         
_________________________________________________________________
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_1 (Batch multiple                  8         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
________________________________________________

Much of the time, however, models which compose many layers simply call one layer after the other. This can be done in very little code using tf.keras.Sequential:

In [26]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1,1),
                                                    input_shape=(None,None,3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2,1,padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1,1)),
                             tf.keras.layers.BatchNormalization()])

my_seq(tf.zeros([1,2,3,3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [27]:
my_seq.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_5 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_6 (Batch (None, None, None, 2)     8         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_7 (Batch (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
_________________________________________________________